In [ ]:
import random
import os
from typing import NamedTuple

import numpy as np
import torch
import torchvision
from torch.utils import data
from PIL import Image

import model
from tensor_transforms import convert_to_coord_format

In [ ]:
class FFHQ256Arguments(NamedTuple):
    """CIPSskip for FFHQ-256"""
    Generator = 'CIPSskip'
    output_dir = 'ffhq256_g_ema.pt'
    out_path = 'checkpoint'
    size = 256
    coords_size = 256
    fc_dim = 512 
    latent = 512
    style_dim = 512
    n_mlp = 8
    activation = None
    channel_multiplier = 2
    ckpt = os.path.join(out_path, output_dir)
    coords_integer_values = False

In [ ]:
class FFHQ1024Arguments(NamedTuple):
    """CIPSskip-progressive for FFHQ-1024"""
    Generator = 'CIPSskip'
    output_dir = 'ffhq1024_g_ema.pt'
    out_path = 'checkpoint'
    size = 256
    coords_size = 1024
    fc_dim = 512 
    latent = 512
    style_dim = 512
    n_mlp = 8
    activation = None
    channel_multiplier = 2
    ckpt = os.path.join(out_path, output_dir)
    coords_integer_values = False

In [ ]:
class Churches256Arguments(NamedTuple):
    """CIPSskip for LSUN-Churches-256"""
    Generator = 'CIPSskip'
    output_dir = 'churches_g_ema.pt'
    out_path = 'checkpoint'
    size = 256
    coords_size = 256
    fc_dim = 512 
    latent = 512
    style_dim = 512
    n_mlp = 8
    activation = None
    channel_multiplier = 2
    ckpt = os.path.join(out_path, output_dir)
    coords_integer_values = False

In [ ]:
class Lanscapes256Arguments(NamedTuple):
    """CIPSres for Landscapes-256"""
    Generator = 'CIPSres'
    output_dir = 'landscapes_g_ema.pt'
    out_path = 'checkpoint'
    size = 256
    coords_size = 256
    fc_dim = 512 
    latent = 512
    style_dim = 512
    n_mlp = 8
    activation = None
    channel_multiplier = 2
    ckpt = os.path.join(out_path, output_dir)
    coords_integer_values = False

In [ ]:
args = Lanscapes256Arguments()
device = 'cuda'

In [ ]:
Generator = getattr(model, args.Generator)
g_ema = Generator(size=args.size, hidden_size=args.fc_dim, style_dim=args.latent, n_mlp=args.n_mlp,
                  activation=args.activation, channel_multiplier=args.channel_multiplier,
                  ).to(device)
g_ema.eval();

In [ ]:
ckpt = torch.load(path)
g_ema.load_state_dict(ckpt)

In [ ]:
def get_image(tensor, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    
    grid = torchvision.utils.make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    return im

## Finding mean for truncation trick 

In [ ]:
n_sample = 1
sample_z = torch.randn(n_sample, args.latent, device=device)
converted_full = convert_to_coord_format(sample_z.size(0), args.coords_size, args.coords_size, device,
                                         integer_values=args.coords_integer_values)

latents = []
samples = []
with torch.no_grad():
    for _ in range(100):
        sample_z = torch.randn(n_sample, args.latent, device=device)
        sample, latent = g_ema(converted_full, [sample_z], return_latents=True)
        latents.append(latent.cpu())
        samples.append(sample.cpu())

samples = torch.cat(samples, 0)
latents = torch.cat(latents, 0)

truncation_latent = latents.mean(0).cuda()

print('truncation_latent', truncation_latent.shape)
assert len(truncation_latent.shape)==1 and truncation_latent.size(0) == 512, 'smt wrong'

## Sampling with truncation trick 

In [ ]:
n_sample = 4
sample_z = torch.randn(n_sample, args.latent, device=device)
converted_full = convert_to_coord_format(sample_z.size(0), args.coords_size, args.coords_size, device,
                                         integer_values=args.coords_integer_values)

with torch.no_grad():
    style = g_ema.style(sample_z)
    sample, _ = g_ema(converted_full, [style], 
                      truncation=0.6,
                      truncation_latent=truncation_latent,
                      input_is_latent=True,)
    
im = get_image(sample,                        
                nrow=int(n_sample ** 0.5),
                normalize=True,
                range=(-1, 1),)

im